In [ ]:
!pip install efficientnet


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '/kaggle/input/aptos2019-blindness-detection/'
df = pd.read_csv(path + 'train.csv', sep = ',')
df.head()


In [ ]:
df['diagnosis'].hist()
df['diagnosis'].value_counts()

In [ ]:
df['diagnosis'].value_counts()

In [ ]:
# burada amaç train_images deki resimlerin isimlerini liste halinde bulmak bunu yapmassak resimleri okuyamayız.
import os
files = os.listdir(path + 'train_images')
len(files)


In [ ]:
import cv2
img_list = []

for i in files[0:20]:
    image = cv2.imread(path + 'train_images/'+i)
    image = cv2.resize(image,(400,400))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_list.append(image)

In [ ]:
plt.imshow(img_list[0])

In [ ]:
plt.imshow(img_list[4])


In [ ]:
kopya = img_list[4].copy()

In [ ]:
kopya = cv2.cvtColor(kopya, cv2.COLOR_RGB2GRAY)


In [ ]:
plt.imshow(kopya, cmap='gray')

In [ ]:
kopya.shape

In [ ]:
blur = cv2.GaussianBlur(kopya,(5,5),0)

In [ ]:
plt.imshow(blur,cmap='gray')

In [ ]:
thresh = cv2.threshold(blur,10,255, cv2.THRESH_BINARY)[1]

In [ ]:
plt.imshow(thresh, cmap='gray')

In [ ]:

kontur = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
kontur = kontur[0][0]
kontur.shape

In [ ]:
kontur = kontur[:,0,:]
kontur.shape

In [ ]:
kontur[:,0].argmax()

In [ ]:
kontur[335]

In [ ]:
kontur[:,0].argmin()


In [ ]:
kontur[111]

In [ ]:

sol = tuple(kontur[kontur[:,0].argmin()])
sağ = tuple(kontur[kontur[:,0].argmax()])
üst = tuple(kontur[kontur[:,1].argmin()])
alt = tuple(kontur[kontur[:,1].argmax()])

In [ ]:
sol, sağ, üst, alt

In [ ]:
    x1 = sol[0]
    y1 = üst[1]
    x2 = sağ[0]
    y2 = alt[1]

In [ ]:
x1, x2, y1, y2

In [ ]:
orijinal = img_list[4].copy()

In [ ]:
plt.imshow(orijinal)


In [ ]:
crop_ilk = orijinal[y1:y2 , x1:x2]

In [ ]:
plt.imshow(crop_ilk)

In [ ]:
crop_ilk.shape

In [ ]:
crop_ilk = cv2.resize(crop_ilk,(400,400))

In [ ]:
plt.imshow(crop_ilk)

In [ ]:
x = int(x2-x1)*4//100
y = int(y2-y1)*5//100

In [ ]:
x,y

In [ ]:
crop_son = orijinal[y1+y : y2-y , x1+x : x2-x]
plt.imshow(crop_son)

In [ ]:
crop_son = cv2.resize(crop_son,(400,400))

In [ ]:
plt.imshow(crop_son)

In [ ]:
lab = cv2.cvtColor(crop_son, cv2.COLOR_RGB2LAB)
lab.shape

In [ ]:
l,a,b = cv2.split(lab)
plt.imshow(l, cmap='gray')

In [ ]:
l.shape
düz = l.flatten()
düz.shape

In [ ]:
plt.hist(düz,25,[0,256], color = 'r')
plt.show()

In [ ]:
clahe = cv2.createCLAHE(clipLimit=7.0,tileGridSize=((8,8)))
cl = clahe.apply(l)

In [ ]:
plt.hist(cl.flatten(),25,[0,256], color = 'r')
plt.show()

In [ ]:
plt.imshow(cl)

In [ ]:
plt.imshow(l)

In [ ]:
limg = cv2.merge((cl,a,b))
son = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
plt.imshow(son)

In [ ]:
plt.imshow(crop_son)

In [ ]:
med_son = cv2.medianBlur(son, 3)
plt.imshow(med_son)

In [ ]:
arka_plan = cv2.medianBlur(son, 37)
plt.imshow(arka_plan)

In [ ]:
maske = cv2.addWeighted(med_son,1,arka_plan,-1,255)
plt.imshow(maske)

In [ ]:
son_img = cv2.bitwise_and(maske,med_son)
plt.imshow(son_img)

In [ ]:
plt.imshow(med_son)

In [ ]:
img_list = []

from tqdm import tqdm_notebook as tqdm

for i in tqdm(files):
    image = cv2.imread(path + 'train_images/'+i)
    image = cv2.resize(image,(400,400))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    kopya = image.copy()
    kopya = cv2.cvtColor(kopya, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(kopya,(5,5),0)
    thresh = cv2.threshold(blur,10,255, cv2.THRESH_BINARY)[1]
    kontur = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    kontur = kontur[0][0]
    kontur = kontur[:,0,:]
    x1 = tuple(kontur[kontur[:,0].argmin()])[0]
    y1 = tuple(kontur[kontur[:,1].argmin()])[1]
    x2 = tuple(kontur[kontur[:,0].argmax()])[0]
    y2 = tuple(kontur[kontur[:,1].argmax()])[1]
    x = int(x2-x1)*4//50
    y = int(y2-y1)*5//50
    kopya2 = image.copy()
    if x2-x1 >100 and y2-y1> 100:
        kopya2 = kopya2[y1+y : y2-y , x1+x : x2-x]
        kopya2 = cv2.resize(kopya2,(400,400))
    lab = cv2.cvtColor(kopya2, cv2.COLOR_RGB2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=5.0,tileGridSize=((8,8)))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    son = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    med_son = cv2.medianBlur(son, 3)
    arka_plan = cv2.medianBlur(son, 37)
    maske = cv2.addWeighted(med_son,1,arka_plan,-1,255)
    son_img = cv2.bitwise_and(maske,med_son)
    img_list.append(son_img)

In [ ]:
plt.imshow(img_list[6])

In [ ]:
fig = plt.figure(figsize=(20,12))

for i in range(12):
    img = img_list[i]
    fig.add_subplot(3,4,i+1)
    plt.imshow(img)

plt.tight_layout()

In [ ]:
y_train = pd.get_dummies(df['diagnosis']).values
df['diagnosis'][1]

In [ ]:
y_train[1]

In [ ]:
y_train_son = np.ones(y_train.shape, dtype='uint8')
y_train_son[:,4] = y_train[:,4]

In [ ]:
np.logical_or(0,0)

In [ ]:
np.logical_or(1,0)

In [ ]:
for i in range(3,-1,-1):
    y_train_son[:,i] = np.logical_or(y_train[:,i], y_train_son[:,i+1])

In [ ]:
y_train_son

In [ ]:
y_train

In [ ]:
x_train = np.array(img_list)
x_train.shape

In [ ]:
y_train_son.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val , y_train, y_val = train_test_split(x_train,
                                                   y_train_son,
                                                   test_size=0.15,
                                                   random_state=2019,
                                                   shuffle=True)

In [ ]:
x_train.shape, x_val.shape , y_train.shape, y_val.shape

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)
data_generator = datagen.flow(x_train,y_train,batch_size=2,seed=2020)

In [ ]:
from efficientnet.keras import EfficientNetB5

In [ ]:
örnek_model = EfficientNetB5()

In [ ]:
# örnek_model.summary()
örnek_model2 = EfficientNetB5(include_top=False)
# örnek_model2.summary()

In [ ]:

from keras.models import Sequential
from keras import layers

model = Sequential()
model.add(EfficientNetB5(weights='imagenet',include_top=False, input_shape=(400,400,3)))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5,activation = 'sigmoid'))

In [ ]:
from keras.optimizers import Adam
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.00005),metrics=['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau

lr = ReduceLROnPlateau(monitor = 'val_loss',
                      patience = 3,
                      verbose = 1,
                      mode='auto',
                      factor=0.25,
                      min_lr=0.000001)

In [ ]:
history = model.fit_generator(data_generator,
                             steps_per_epoch = 1000,
                             epochs = 1,
                             validation_data = (x_val,y_val),
                             callbacks = [lr])